In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

In [2]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/' + 'master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',' )

In [3]:
cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


In [4]:
# Precisameos Iterar dentro de um intervalo de tempo definido para extraí-lo.
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
    date_range_days: int = (end_date - start_date).days
    for lag in range(date_range_days):
        yield start_date + timedelta(lag)

In [6]:
start_date = datetime(2021, 1, 1)
end_date = datetime(2021, 12, 31)

In [7]:
# Selecionar as colunas de interesse e linhas referentes ao Brasil

cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):
    date_str = date.strftime('%m-%d-%Y')
    data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'
    case = pd.read_csv(data_source_url, sep=',')
    
    case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Case_Fatality_Ratio'], axis=1)
    case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
    case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))
    
    if cases_is_empty:
        cases = case
        cases_is_empty = False
    else:
        cases = pd.concat([cases, case], ignore_index=True)

In [8]:
# Consultando o Estado de São Paulo

cases.query('Province_State == "Sao Paulo"').head()

,Province_State,Country_Region,Confirmed,Deaths,Combined_Key,Incident_Rate,Date
24,Sao Paulo,Brazil,1466191,46775,"Sao Paulo, Brazil",3192.990778,2021-01-01
51,Sao Paulo,Brazil,1467953,46808,"Sao Paulo, Brazil",3196.827966,2021-01-02
78,Sao Paulo,Brazil,1471422,46845,"Sao Paulo, Brazil",3204.382565,2021-01-03
105,Sao Paulo,Brazil,1473670,46888,"Sao Paulo, Brazil",3209.278136,2021-01-04
132,Sao Paulo,Brazil,1486551,47222,"Sao Paulo, Brazil",3237.329676,2021-01-05


In [9]:
# Visualização
cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Combined_Key,Incident_Rate,Date
0,Acre,Brazil,41689,796,"Acre, Brazil",4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,"Alagoas, Brazil",3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,"Amapa, Brazil",8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,"Amazonas, Brazil",4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,"Bahia, Brazil",3326.039611,2021-01-01


In [ ]:
cases.shape

In [ ]:
# Informações sobre os dados / Schemas
cases.info()

In [ ]:
cases = cases.rename(
    columns={
        'Province_State': 'state',
        'Country_Region': 'country'
    }
)


for col in cases.columns:
    cases = cases.rename(columns={col: col.lower()})

In [ ]:
cases.head()

In [ ]:
# Ajustando o nome dos estados.
states_map = {
    'Amapa': 'amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo',
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

In [ ]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year'] = cases['date'].apply(lambda date: date.strftime('%Y'))

In [ ]:
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

In [ ]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

    if np.isnan(rate):
        return np.NaN

    if rate < 0.75:
        status = 'downward'
    elif rate > 1.15:
        status = 'upward'
    else:
        status = 'stable'

    return status



for state in cases['state'].drop_duplicates():

    cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
    cases_per_state = cases_per_state.sort_values(by=['date'])


    cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
    cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
    cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
    cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

    cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
    cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
    cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
    cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

    if cases_is_empty:
        cases_ = cases_per_state
        cases_is_empty = False
    else:
        cases_ = pd.concat([cases_, cases_per_state], ignore_index=True)

cases = cases_
cases_ = None
    

In [ ]:
# Garantir o tipo do dado é fundamental para a consistência da base de dados. Vamos fazero type casting das colunas.
cases['population'] = cases['population'].astype('Int32')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int32')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int32')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int32')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int32')

In [ ]:
#Reorganizando as colunas e conferindo o resultado final.
cases = cases[['date','country', 'state', 'population', 'confirmed', 'confirmed_1d','confirmed_moving_avg_7d',
               'confirmed_moving_avg_7d_rate_14d','confirmed_trend', 'deaths_1d', 'deaths_moving_avg_7d',
               'deaths_moving_avg_7d_rate_14d','deaths_trend','month', 'year'
              ]]

In [ ]:
cases.head(n=25)

In [ ]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)

In [ ]:
#Salvando arquivo num DataFrame.
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3])

In [ ]:
# Visualização
vaccines.head()

In [ ]:
# Vamos selecionar as colunas de interesse e as linhas referentes ao Brasil
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['date','location', 'population', 'total_vaccinations','people_vaccinated','people_fully_vaccinated', 'total_boosters',]]

In [ ]:
vaccines.head()

In [ ]:
vaccines.info()

In [ ]:
vaccines = vaccines.fillna(method='ffill')

In [ ]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

In [ ]:
# Agora vamos alterar o nome das colunas.
vaccines = vaccines.rename(
    columns={
        'location': 'country',
        'total_vaccinations': 'total',
        'people_vaccinated': 'one_shot',
        'people_fully_vaccinated': 'two_shots',
        'total_boosters': 'three_shots',
    }
)

In [ ]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year'] = vaccines['date'].apply(lambda date: date.strftime('%Y'))

In [ ]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

In [ ]:
# Garantir o tipo do dado é fundamental para a consistência da base de dados. Vamos fazer o type casting das colunas.
vaccines['population'] = vaccines['population'].astype('Int32')
vaccines['total'] = vaccines['total'].astype('Int32')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int32')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int32')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int32')

In [ ]:
# vamos reorganizar as colunas e conferir o resultado final
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots','two_shots_perc', 'three_shots' ,'three_shots_perc', 'month', 'year']]

In [ ]:
vaccines.tail()

In [ ]:
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)